# VacationPy
----

#### Description

This notebook imports the weather data generated by the WeatherPy notebook and creates a Google Maps heatmap for that data. Cities are then selected using the following criteria.

* Max temperature between 65 and 85 degrees F.
* Relative humidity between 50 and 70%.
* Cloud cover less than 30%.

For this selection, a search is conducted for the nearest hotels in those areas. If no hotel is found, then that city is removed from the list.

A second heatmap is produced that overlays the locations of the hotels onto the heatmap.

Viola! Prime vacation locations! A further analysis may need to be conducted to ensure that all locations chosen are in areas devoid of political unrest, or other undesireable characteristics, since weather is not the only factor to be considered when choosing a vacation spot.

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
from api_keys import geo_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
path = 'output_data/cities.csv'
city_data_df = pd.read_csv(path, encoding='utf-8')
# city_test = pd.DataFrame('/output_data/cities.csv')
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acapulco,16.8634,-99.8901,85.82,70,75,8.05,MX,1669584790
1,1,sao joao da barra,-21.6403,-41.0511,75.29,93,91,19.35,BR,1669584933
2,2,codrington,-38.2667,141.9667,52.23,81,100,11.54,AU,1669584934
3,3,barrow,71.2906,-156.7887,3.22,85,100,14.97,US,1669584935
4,4,avera,33.1940,-82.5271,72.91,58,4,15.90,US,1669584936


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df['Humidity'].astype(float)


In [4]:
# Create a weather Heatmap layer
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='500px', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal = city_data_df.loc[((city_data_df['Humidity'] < 70) &
                          (city_data_df['Humidity'] > 50) &
                          (city_data_df['Max Temp'] > 65.0) & 
                          (city_data_df['Max Temp'] < 85.0) & 
                          (city_data_df['Cloudiness'] < 30))]
# ideal.count()
ideal =ideal.dropna()
ideal.count()

City_ID       22
City          22
Lat           22
Lng           22
Max Temp      22
Humidity      22
Cloudiness    22
Wind Speed    22
Country       22
Date          22
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True)
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,avera,33.1940,-82.5271,72.91,58,4,15.90,US,1669584936,
28,28,half moon bay,37.4636,-122.4286,71.58,69,0,10.36,US,1669584950,
33,33,avarua,-21.2078,-159.7750,80.65,65,20,12.66,CK,1669584952,
76,76,sao filipe,14.8961,-24.4956,77.61,64,0,14.20,CV,1669584983,
105,105,cape town,-33.9258,18.4232,69.39,65,0,17.00,ZA,1669584763,
132,132,kavieng,-2.5744,150.7967,84.29,67,10,9.37,PG,1669585021,
162,162,dwarka,22.2394,68.9678,74.55,53,0,6.98,IN,1669585043,
169,169,cabo san lucas,22.8909,-109.9124,83.61,52,0,4.00,MX,1669584469,
188,188,lompoc,34.6391,-120.4579,67.84,52,0,10.36,US,1669585061,
199,199,sur,22.5667,59.5289,76.35,63,28,6.11,OM,1669585067,


In [7]:
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    print(target_coordinates)

33.194, -82.5271
37.4636, -122.4286
-21.2078, -159.775
14.8961, -24.4956
-33.9258, 18.4232
-2.5744, 150.7967
22.2394, 68.9678
22.8909, -109.9124
34.6391, -120.4579
22.5667, 59.5289
21.3469, -158.0858
28.6585, -17.9182
32.3754, 15.0925
-38.7196, -62.2724
20.2342, -77.2789
23.2667, 68.8333
21.3906, -158.1547
30.4091, -89.0436
-21.3393, 55.4781
2.8197, -60.6733
28.017, -97.5169
21.3156, -158.0072


In [18]:
hotel_name = []
hotel_lat = []
hotel_lng = []

print("Starting hotel search")

for index, row in hotel_df.iterrows():
# target_coordinates = [row['Lat'], row["Lng"]]
# geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_search = "hotel"
    target_radius = 10000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    my_request = requests.get(base_url, params=params).json()
    try:
        hotel_name.append(my_request['results'][0]['name'])
        hotel_lat.append(my_request['results'][0]['geometry']['location']['lat'])
        hotel_lng.append(my_request['results'][0]['geometry']['location']['lng'])
        
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


print('---------------------')
print('Hotel Search Complete')


    


Starting hotel search
half moon bay - nearest hotel: 
avarua - nearest hotel: 
sao filipe - nearest hotel: 
cape town - nearest hotel: 
kavieng - nearest hotel: 
dwarka - nearest hotel: 
cabo san lucas - nearest hotel: 
lompoc - nearest hotel: 
sur - nearest hotel: 
makakilo city - nearest hotel: 
los llanos de aridane - nearest hotel: 
misratah - nearest hotel: 
bahia blanca - nearest hotel: 
naliya - nearest hotel: 
nanakuli - nearest hotel: 
harrison - nearest hotel: 
saint-pierre - nearest hotel: 
boa vista - nearest hotel: 
san patricio - nearest hotel: 
ewa beach - nearest hotel: 
---------------------
Hotel Search Complete


In [20]:
fix_hotel = hotel_df.loc[hotel_df["Hotel Name"] != 'No hotel found']
hotel_df = fix_hotel
hotel_df['Hotel Name'] = hotel_name
hotel_df['Hotel Lat'] = hotel_lat
hotel_df['Hotel Lng'] = hotel_lng
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
28,28,half moon bay,37.4636,-122.4286,71.58,69,0,10.36,US,1669584950,Half Moon Bay Lodge,37.433209,-122.428202
33,33,avarua,-21.2078,-159.7750,80.65,65,20,12.66,CK,1669584952,Little Polynesian Resort,-21.272277,-159.751565
76,76,sao filipe,14.8961,-24.4956,77.61,64,0,14.20,CV,1669584983,Hotel Santos Pina,14.892737,-24.498085
105,105,cape town,-33.9258,18.4232,69.39,65,0,17.00,ZA,1669584763,O’Two Boutique Hotel,-33.900387,18.405447
132,132,kavieng,-2.5744,150.7967,84.29,67,10,9.37,PG,1669585021,Nusa Island Retreat,-2.582447,150.781006
162,162,dwarka,22.2394,68.9678,74.55,53,0,6.98,IN,1669585043,Dwarkadhish Lords Eco Inn,22.238970,68.961838
169,169,cabo san lucas,22.8909,-109.9124,83.61,52,0,4.00,MX,1669584469,Solmar Resort,22.876556,-109.903533
188,188,lompoc,34.6391,-120.4579,67.84,52,0,10.36,US,1669585061,O'Cairns Inn & Suites,34.638329,-120.446843
199,199,sur,22.5667,59.5289,76.35,63,28,6.11,OM,1669585067,Sur Grand Hotel,22.600993,59.481204
204,204,makakilo city,21.3469,-158.0858,79.45,56,0,9.22,US,1669585070,Hampton Inn & Suites Oahu/Kapolei,21.332245,-158.052334


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(0, 0), zoom_level=2.20, layout={'width': '1200px', 'height': '500px'})
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='1200px'))

<Figure size 640x480 with 0 Axes>